# Example: Memory Representation of Floating Point Numbers
Floating point numbers $x\in\mathbb{R}$ are stored using 4$\times$bytes (single-precision) or 8$\times$bytes (double-precision)
following the [IEEE-754 standard](https://en.wikipedia.org/wiki/IEEE_754). 
The components of number $x\in\mathbb{R}$ are encoded in different segments of the`32-bit` or `64-bit` word, e.g., for a `64-bit` float:
\begin{equation*}
x = -1^{S}\times{M}\times{2}^{(E-1023)}
\end{equation*}
where $S$ denotes the sign bit, $M$ denotes the mantissa (fraction) and $E$ denotes the exponent.
* For a `32-bit` floating point number, $S$ is bit 31 denoted by $b_{31}$, $M$ is encoded in bits $b_0\rightarrow{b_{22}}$ and $E$ is encoded by bits $b_{23}\rightarrow{b_{30}}$.
* For a `64-bit` floating point number, the sign bit $S$ is $b_{63}$, the mantissa $M$ is the number encoded by bits $b_0\rightarrow{b_{51}}$, and the exponent $E$ is encoded by bits $b_{52}\rightarrow{b_{62}}$.

Let's specify a `64-bit` floating point number in the variable `floating_point_nunber_example` and then reconstruct that number from the bit pattern:

In [12]:
floating_point_nunber_example = 2.311898736;

We know we can get the bit pattern by calling the `bitstring(...)` function, but the wrinkle is that `bitstring(...)` returns the bit pattern as a `String.` We'll have to convert the `String` to an array of `0` and `1`, but more on that later.

In [2]:
bitstring(floating_point_nunber_example)

"0100000000000010011111101100010011000011110000011100100011110001"

To understand where to go from here, we need a few things (the following `code` line contains a few advanced things, e.g., working with arrays and `String` and `Char` types, function piping, etc; don't worry too much about these now):
* First, we can convert a string into an array, i.e., an ordered list of `Char`s using the [collect function](https://docs.julialang.org/en/v1/base/collections/#Base.collect-Tuple{Type,%20Any}). If we have the `Char` version of `1` or `0`, we can try to convert it into an `Int` using the [parse function](https://docs.julialang.org/en/v1/base/numbers/#Base.parse) in Julia.
    * _Aside_: in older languages such as `C,` strings were natively represented as arrays of characters. However, newer languages that include [Unicode character sets](https://en.wikipedia.org/wiki/Unicode) have dedicated `String` types that are more complex. 
* Next, we can string together commands, i.e., `cmd_1 |> cmd_2` where the output of `cmd_1` is the input into `cmd_2` using the [Julia `|>` piping operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping)

In [3]:
bit_pattern_array = bitstring(floating_point_nunber_example) |> collect |> reverse .|> x-> parse(Int,x)

64-element Vector{Int64}:
 1
 0
 0
 0
 1
 1
 1
 1
 0
 0
 0
 1
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 1
 0

`Hack`: fix the `1`-based array issue and introduce an important (and supremely useful) data structure called a [Dictionary](https://docs.julialang.org/en/v1/base/collections/#Base.Dict) that we can use to make a `0`-based array (sort of). We'll copy the `bit_pattern_array` into a dictionary (which we can make `0`-based), called `bit_patten_dictionary`:

In [4]:
bit_patten_dictionary = Dict{Int64,Int64}(); # what are we doing here?
for i ∈ eachindex(bit_pattern_array)
    bit_patten_dictionary[i-1] = bit_pattern_array[i] # what are we doing here?
end
bit_patten_dictionary; # what is going on here?

## Sign bit `S`
The `sign` bit for a `64-bit` number is in position `63`, i.e., $b_{63}$. Let's look up that value in the `bit_patten_dictionary` and set it equal to the variable `S.` 
* We need to compute the sign manually because of the weirdness about zero exponents and store this value in the `sign` variable. The default value `sign = 1` (positive). However, if the `sign` bit is `1`, i.e., $b_{63} = 1$, we set `sign = -1` (negative).

In [5]:
sign = 1;
S = bit_patten_dictionary[63]
if (S == 1)
    sign = -1;
end
sign

1

## Mantissa `M`

In [6]:
mantissa_bit_range = range(1,stop=52,step = 1) |> collect;

In [7]:
M = 1.0;
base = 2.0;
for k ∈ eachindex(mantissa_bit_range)
    i = mantissa_bit_range[k]
    aₖ = bit_patten_dictionary[52-i]
    M += aₖ*(base^(-i))
end

In [8]:
M

1.1559493681675657

## Exponent

In [9]:
exponent_bit_range = range(52,stop=62, step = 1) |> collect;

In [10]:
E = 0.0
base = 2;
for k ∈ eachindex(exponent_bit_range)   
    index = exponent_bit_range[k]
    aₖ = bit_patten_dictionary[index]    
    E += aₖ*(base^(k-1))
end
E

1024.0

### Value

In [11]:
x = (sign)*M*(2^(E - 1023))

2.3118987363351313